In [1]:
!pip3 install -q transformers

     |████████████████████████████████| 3.8 MB 5.2 MB/s 
     |████████████████████████████████| 67 kB 6.5 MB/s 
     |████████████████████████████████| 895 kB 58.2 MB/s 
     |████████████████████████████████| 6.5 MB 55.9 MB/s 
     |████████████████████████████████| 596 kB 55.3 MB/s 


# 330. Transformers 기타 Model들


## T5 pre-trained model

In [2]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration

tokenizer = T5TokenizerFast.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [3]:
def t5_task(text):
    input_ids = tokenizer.encode(text, return_tensors='pt')
    outputs = model.generate(input_ids)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

### 기계 번역

In [4]:
text = "translate English into German: I love you."
t5_task(text)

'Ich liebe dich.'

In [5]:
text = "translate English into French: I love you."
t5_task(text)

'Je vous aime.'

### CoLA (The Corpus of Linguistic Acceptability)
문장이 문법적으로 맞는지 분류 (이진분류)

In [6]:
text = "cola sentence: I bought fruits and."
t5_task(text)

'unacceptable'

In [7]:
text = "cola sentence: I bought fruits and vegetables."
t5_task(text)

'acceptable'

### 질의 응답

In [8]:
text = "question: Which is the highest mountain in the world ?"
t5_task(text)

'Mount Everest'

### STS-B (The Semantic Textual Similarity Benchmark)
문장 A 와 B 는 얼마나 유사한가 ?

In [9]:
text = "stsb sentence1: Cats and dogs are mammals. sentence2: There are four forces in physics."
t5_task(text)

'0.0'

In [10]:
text = "stsb sentence1: Cats and dogs are animals. sentence2: Cats and dogs are pets."
t5_task(text)

'5.0'

### 문장 요약

In [11]:
text = """summarize: The t5 library serves primarily as code for reproducing the experiments in Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer. In the paper, we demonstrate how to achieve state-of-the-art results on multiple NLP tasks using a text-to-text transformer pre-trained on a large text corpus.
The bulk of the code in this repository is used for loading, preprocessing, mixing, and evaluating datasets. It also provides a way to fine-tune the pre-trained models released alongside the publication.
The t5 library can be used for future model development by providing useful modules for training and fine-tuning (potentially huge) models on mixtures of text-to-text tasks.
"""
t5_task(text)

'the t5 library is used for reproducing the experiments in Exploring the Limits of'

## 한국어 pre-trained model

https://github.com/kiyoungkim1/LMkor

## 한국어 text 생성

In [12]:
from transformers import BertTokenizerFast, GPT2LMHeadModel

tokenizer = BertTokenizerFast.from_pretrained('kykim/gpt3-kor-small_based_on_gpt2')
model = GPT2LMHeadModel.from_pretrained('kykim/gpt3-kor-small_based_on_gpt2', pad_token_id=0)

Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

In [13]:
text = "오늘은 왠지"

input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = input_ids[:, 1:]  # remove cls token
outputs = model.generate(input_ids)
tokenizer.decode(outputs[0], skip_special_tokens=True)

'오늘은 왠지 더 잘할 수 있을 것 같은 느낌이 들어요.'

## 한국어 문서 요약

In [14]:
from transformers import BertTokenizerFast, EncoderDecoderModel

tokenizer = BertTokenizerFast.from_pretrained('kykim/bertshared-kor-base')
model = EncoderDecoderModel.from_pretrained('kykim/bertshared-kor-base')

text = '''
LG전자가 스마트폰을 담당하는 MC(모바일커뮤니케이션)사업부 분할 및 매각을 위한 법률 자문 업무를 김앤장법률사무소에 맡겼다. 
MC사업부 매각 작업에 속도가 붙을지 관심이 집중되고 있다.
22일 인수합병(M&A)업계에 따르면 LG전자는 최근 MC사업부 분할 후 매각 방안 등을 포괄적으로 검토하기 위해 김앤장을 법률자문사로 
선임한 것으로 알려졌다. 
회계·실사 자문은 EY한영회계법인에 맡길 가능성이 큰 것으로 전해졌다. 김앤장 등 자문사들은 사업본부를 분할한 뒤 사업양수도나 분할사업부의 지분 매각, 
지식재산권(IP) 매각 등을 놓고 검토에 들어간 것으로 알려졌다.
업계에서는 LG전자가 MC사업본부를 통매각하기보다는 ‘쪼개기 매각’에 나설 것으로 보고 있다. 스마트폰 선행기술 연구개발(R&D) 등 핵심 기능만 남겨둔 채 
매각을 시도할 것으로 관측하고 있다. 앞서 권봉석 LG전자 사장은 사내 메시지를 통해 임직원에게 “현재 모든 가능성을 열어 두고 사업 운영방향을 면밀히 
검토하고 있다”고 밝히며 매각 추진을 암시했다. M&A업계 관계자는 “거래가 성사되기도 전에 사업 전면 재검토를 공식화한 것은 상당히 이례적”이라며 
“향후 매각이 잘 이뤄지지 않더라도 모바일 사업을 철수하겠다는 배수진을 둔 것으로 보인다”고 설명했다.
다만 원매자를 찾기가 쉽지 않을 것이란 전망이 우세하다. LG전자 모바일 사업은 한때 글로벌시장에서 톱5 안에 드는 기술력을 인정받았지만 
누적 적자만 5조원에 달하고 있다. 업계에서 평가하는 MC사업부의 가치도 5000억원대에서 수조원대까지 편차가 상당히 크다.
상대적으로 해외 원매자들의 인수의사가 더 확실한 것으로 알려지고 있다. 북미사업 등 글로벌 시장 확장을 원하는 후발기업들이 주요 대상이다. 
베트남의 빈그룹과 중국 기업 등이 유력하게 거론된다. 증권업계를 중심으로는 스마트 기기를 연결하는 사물인터넷(IoT) 사업을 염두에 둔 구글, 
페이스북 같은 미국 정보기술(IT) 기업들도 원매자 후보군으로 꼽고 있다.
'''

input_ids = tokenizer.encode(text, return_tensors= 'pt')
sentence_length = len(input_ids[0])
print(sentence_length)

outputs = model.generate(
            input_ids,
            min_length=max(10, int(0.1*sentence_length)),
            max_length=min(100, int(0.3*sentence_length))
        )
            
tokenizer.decode(outputs[0], skip_special_tokens=True)

Downloading:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/562M [00:00<?, ?B/s]

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


412


'22일 인수합병 ( m & a ) 업계에 따르면 스마트폰 선행기술 연구개발 ( r & d ) 등 핵심 기능만 남겨둔 채 매각을 시도할 것으로 관측되고 있으며 업계에서는 mc사업부 매각 작업에 속도가 붙을지 관심이 집중되고 있다.'